In [1]:
import numpy as np
import pandas as pd

from utils import load_config

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')

config = load_config()

In [2]:
def df_to_latex(df, caption: str = None):
    
    result = """\
\\begin{{table}}[H]
\t\\resizebox{{\\textwidth}}{{!}}{{
\t\t\\begin{{tabular}}{{|>{{\\bfseries}}l|{}}}
\t\t\t\\hline
{}
\t\t\\end{{tabular}}
\t}}{}
\\end{{table}}\
"""

    columns = 'c|' * df.shape[1]
        
    body = ['& ' + ' & '.join(df.columns)]
    
    body += [
        df.index[i] +
        '&' + 
        ' & '.join(map(lambda x: f'${np.mean(x):.3f} \\pm {np.std(x):.3f}$', output.iloc[i].values)) 
        for i in range(len(df))
    ]
    
    for i in range(len(body)):
        body[i] = '\t' * 3 + body[i] + ' \\\\ \\hline'
        

    body = '\n'.join(body).replace('_', '\\_')
    
    if caption is not None:
        caption = f'\caption{{{caption}}}'
    else:
        caption = ''

    result = result.format(columns, body, caption)

    return result

In [3]:
model_names = [
    'xlm-r',
    'xlm-r adv pretrained',
    'xlm-r pretrained',
    'm-bert',
    'm-bert adv pretrained',
    'm-bert pretrained'
]

model_args = [
    ('xlm-r', False, False),
    ('xlm-r', False, True),
    ('xlm-r', True, False),
    ('m-bert', False, False),
    ('m-bert', False, True),
    ('m-bert', True, False)
]

In [4]:
def get_model_attacks(language, model_name, pretrained: bool = False, load_body: bool = False):
    result = pd.read_csv(f'results/{language}/{model_name}_{int(pretrained)}_{int(load_body)}.csv', index_col=0)

    return result

In [5]:
get_model_attacks('en', 'm-bert')

,intent_acc,slot_f1,sementic_frame_acc,loss
No attack,0.968212,0.932387,0.765563,0.415349
Word level [de],0.839735,0.572102,0.107285,3.867909
Word level [es],0.790728,0.347826,0.029139,6.526373
Word level [fr],0.768212,0.347925,0.022517,7.653117
Word level [ja],0.712583,0.430544,0.015894,4.274644
Word level [pt],0.777483,0.319301,0.005298,7.036911
Word level [zh],0.798675,0.499954,0.054305,3.639285
Alignments [de],0.912583,0.736660,0.347020,2.064145
Alignments [es],0.919205,0.761114,0.336424,1.876161
Alignments [fr],0.915232,0.701310,0.246358,2.450314


In [6]:
# СРАВНЕНИЕ МОДЕЛЕЙ МЕЖДУ СОБОЙ ПРОСТО НА ТЕСТОВОЙ ВЫБОРКЕ

output = {}

for model_name, model_arg in zip(model_names, model_args):
    results = {'intent_acc': [], 'slot_f1': [], 'sementic_frame_acc': [], 'loss': []}

    for language in config['languages']:
        df = get_model_attacks(language, *model_arg)

        for key in results.keys():
            results[key].append(df.loc['No attack'][key])

    output[model_name] = results
    
output = pd.DataFrame.from_dict(output)

output

,xlm-r,xlm-r adv pretrained,xlm-r pretrained,m-bert,m-bert adv pretrained,m-bert pretrained
intent_acc,"[0.9682119205298012, 0.952317880794702, 0.9165...","[0.9655629139072848, 0.96158940397351, 0.92450...","[0.9748344370860929, 0.976158940397351, 0.9562...","[0.9682119205298012, 0.9602649006622516, 0.952...","[0.970860927152318, 0.96158940397351, 0.958940...","[0.9774834437086092, 0.9721854304635762, 0.972..."
slot_f1,"[0.9274118738404452, 0.921545319465082, 0.8227...","[0.9393410175806264, 0.9291455790413816, 0.806...","[0.955545265107664, 0.9471189453414308, 0.9047...","[0.9323868264866751, 0.9151291512915128, 0.808...","[0.936333062739186, 0.9181905678537056, 0.8047...","[0.9497103128621088, 0.9491416653296968, 0.885..."
sementic_frame_acc,"[0.7509933774834437, 0.7602649006622516, 0.445...","[0.7801324503311259, 0.7841059602649006, 0.437...","[0.8649006622516556, 0.8609271523178808, 0.696...","[0.7655629139072848, 0.7443708609271523, 0.422...","[0.8026490066225166, 0.7748344370860927, 0.456...","[0.8543046357615894, 0.8622516556291391, 0.667..."
loss,"[0.4442028044557542, 0.5650126276286564, 0.888...","[0.3948793159062533, 0.4912045914817626, 0.830...","[0.3003557605773063, 0.3667781513539682, 0.528...","[0.4153490327519217, 0.4108025485472018, 0.791...","[0.390667095019384, 0.4297191020240701, 0.8075...","[0.3610887048012381, 0.3643788474951993, 0.540..."


In [7]:
print(df_to_latex(output, 'Таблица сравнения моделей между собой на тестовой выборке'))

\begin{table}[H]
	\resizebox{\textwidth}{!}{
		\begin{tabular}{|>{\bfseries}l|c|c|c|c|c|c|}
			\hline
			& xlm-r & xlm-r adv pretrained & xlm-r pretrained & m-bert & m-bert adv pretrained & m-bert pretrained \\ \hline
			intent\_acc&$0.943 \pm 0.016$ & $0.945 \pm 0.020$ & $0.970 \pm 0.007$ & $0.954 \pm 0.010$ & $0.960 \pm 0.007$ & $0.971 \pm 0.008$ \\ \hline
			slot\_f1&$0.898 \pm 0.035$ & $0.903 \pm 0.043$ & $0.937 \pm 0.016$ & $0.895 \pm 0.040$ & $0.898 \pm 0.041$ & $0.933 \pm 0.021$ \\ \hline
			sementic\_frame\_acc&$0.669 \pm 0.098$ & $0.687 \pm 0.109$ & $0.815 \pm 0.054$ & $0.660 \pm 0.104$ & $0.684 \pm 0.104$ & $0.801 \pm 0.066$ \\ \hline
			loss&$0.619 \pm 0.131$ & $0.589 \pm 0.154$ & $0.395 \pm 0.066$ & $0.518 \pm 0.122$ & $0.509 \pm 0.127$ & $0.397 \pm 0.068$ \\ \hline
		\end{tabular}
	}\caption{Таблица сравнения моделей между собой на тестовой выборке}
\end{table}


In [8]:
# СРАВНЕНИЕ КАЧЕСТВА ПО ЯЗЫКАМ НА ТЕСТОВОЙ ВЫБОРКЕ

output = {}

for language in config['languages']:
    results = {'intent_acc': [], 'slot_f1': [], 'sementic_frame_acc': [], 'loss': []}
    
    for model_name, model_arg in zip(model_names, model_args):

        df = get_model_attacks(language, *model_arg)

        for key in results.keys():
            results[key].append(df.loc['No attack'][key])

    output[language] = results

output = pd.DataFrame.from_dict(output)

output

,en,de,es,fr,ja,pt,zh
intent_acc,"[0.9682119205298012, 0.9655629139072848, 0.974...","[0.952317880794702, 0.96158940397351, 0.976158...","[0.9165562913907284, 0.924503311258278, 0.9562...","[0.9562913907284768, 0.9655629139072848, 0.966...","[0.9271523178807948, 0.9417218543046356, 0.978...","[0.9403973509933776, 0.9112582781456954, 0.964...","[0.943046357615894, 0.9456953642384106, 0.9695..."
slot_f1,"[0.9274118738404452, 0.9393410175806264, 0.955...","[0.921545319465082, 0.9291455790413816, 0.9471...","[0.8227272727272726, 0.8060192210419829, 0.904...","[0.8896744734128738, 0.8914352872043486, 0.937...","[0.9090389343090732, 0.9129640205596802, 0.931...","[0.8880045480386584, 0.9034009717062018, 0.933...","[0.9304073552110596, 0.938057848466546, 0.9526..."
sementic_frame_acc,"[0.7509933774834437, 0.7801324503311259, 0.864...","[0.7602649006622516, 0.7841059602649006, 0.860...","[0.4450331125827814, 0.4370860927152318, 0.696...","[0.6966887417218544, 0.713907284768212, 0.8291...","[0.6463576158940397, 0.6688741721854304, 0.785...","[0.6847682119205298, 0.6980132450331126, 0.823...","[0.6993377483443709, 0.7271523178807947, 0.845..."
loss,"[0.4442028044557542, 0.3948793159062533, 0.300...","[0.5650126276286564, 0.4912045914817626, 0.366...","[0.8886856232426669, 0.8301840033284144, 0.528...","[0.5345613523282258, 0.4601895630037976, 0.354...","[0.6873636607785317, 0.6182308186593123, 0.408...","[0.5809180688658288, 0.7899326560703278, 0.428...","[0.628902015403391, 0.53663447222942, 0.376966..."


In [9]:
print(df_to_latex(output, 'Таблица сравнения качества для языков на тестовой выборке'))

\begin{table}[H]
	\resizebox{\textwidth}{!}{
		\begin{tabular}{|>{\bfseries}l|c|c|c|c|c|c|c|}
			\hline
			& en & de & es & fr & ja & pt & zh \\ \hline
			intent\_acc&$0.971 \pm 0.004$ & $0.964 \pm 0.008$ & $0.947 \pm 0.020$ & $0.967 \pm 0.007$ & $0.951 \pm 0.016$ & $0.946 \pm 0.020$ & $0.954 \pm 0.012$ \\ \hline
			slot\_f1&$0.940 \pm 0.010$ & $0.930 \pm 0.013$ & $0.839 \pm 0.041$ & $0.903 \pm 0.023$ & $0.922 \pm 0.010$ & $0.905 \pm 0.019$ & $0.937 \pm 0.011$ \\ \hline
			sementic\_frame\_acc&$0.803 \pm 0.043$ & $0.798 \pm 0.047$ & $0.521 \pm 0.115$ & $0.742 \pm 0.063$ & $0.693 \pm 0.051$ & $0.736 \pm 0.060$ & $0.742 \pm 0.071$ \\ \hline
			loss&$0.384 \pm 0.045$ & $0.438 \pm 0.071$ & $0.731 \pm 0.142$ & $0.433 \pm 0.070$ & $0.535 \pm 0.095$ & $0.529 \pm 0.136$ & $0.481 \pm 0.092$ \\ \hline
		\end{tabular}
	}\caption{Таблица сравнения качества для языков на тестовой выборке}
\end{table}


In [10]:
# СРАВНЕНИЕ ЯЗЫКОВ ДЛЯ АТАКИ WORD LEVEL

output = {}

for language in config['languages']:
    results = {'intent_acc': [], 'slot_f1': [], 'sementic_frame_acc': [], 'loss': []}

    for model_name, model_arg in zip(model_names, model_args):

        df = get_model_attacks(language, *model_arg)

        for key in results.keys():
            for idx, row in df.iterrows():
                if 'Word level' in idx:
                    results[key].append(row[key])

    output[language] = results
    
output = pd.DataFrame.from_dict(output)

output

,en,de,es,fr,ja,pt,zh
intent_acc,"[0.8582781456953642, 0.7933774834437086, 0.792...","[0.8649006622516556, 0.8503311258278146, 0.870...","[0.866225165562914, 0.8675496688741722, 0.8370...","[0.8145695364238411, 0.8357615894039735, 0.860...","[0.8132450331125828, 0.7642384105960265, 0.784...","[0.866225165562914, 0.8264900662251655, 0.8238...","[0.8291390728476821, 0.8596026490066225, 0.839..."
slot_f1,"[0.5721938775510204, 0.403781979977753, 0.3809...","[0.49231591607644, 0.384316581694586, 0.455334...","[0.1371329379430059, 0.5054915254237287, 0.391...","[0.2263588979895756, 0.5592618878637331, 0.411...","[0.1604865025215069, 0.2728862973760932, 0.230...","[0.1820017306028266, 0.5344059761528517, 0.427...","[0.1460264900662251, 0.4249753902404725, 0.362..."
sementic_frame_acc,"[0.0900662251655629, 0.0278145695364238, 0.027...","[0.0344370860927152, 0.0079470198675496, 0.003...","[0.0, 0.0331125827814569, 0.0304635761589403, ...","[0.0105960264900662, 0.1139072847682119, 0.014...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.119205298013245, 0...","[0.0105960264900662, 0.0940397350993377, 0.015...","[0.0013245033112582, 0.0, 0.0, 0.0026490066225..."
loss,"[3.832766835940512, 6.568402860039159, 7.69375...","[7.707464142849571, 8.228081412064402, 8.04486...","[9.665287133267052, 4.523070861163892, 5.56669...","[9.415385672920628, 4.952812143375999, 8.07482...","[16.619856754102205, 16.15186558773643, 16.282...","[9.679460786518298, 5.179689445934797, 6.31074...","[17.09704391077945, 15.664015569184956, 16.342..."


In [11]:
print(df_to_latex(output, 'Таблица сравнения качества по языкам для атаки Word level'))

\begin{table}[H]
	\resizebox{\textwidth}{!}{
		\begin{tabular}{|>{\bfseries}l|c|c|c|c|c|c|c|}
			\hline
			& en & de & es & fr & ja & pt & zh \\ \hline
			intent\_acc&$0.818 \pm 0.058$ & $0.858 \pm 0.063$ & $0.846 \pm 0.053$ & $0.842 \pm 0.056$ & $0.815 \pm 0.037$ & $0.829 \pm 0.060$ & $0.866 \pm 0.032$ \\ \hline
			slot\_f1&$0.483 \pm 0.136$ & $0.427 \pm 0.088$ & $0.362 \pm 0.146$ & $0.418 \pm 0.124$ & $0.283 \pm 0.142$ & $0.420 \pm 0.142$ & $0.390 \pm 0.142$ \\ \hline
			sementic\_frame\_acc&$0.073 \pm 0.081$ & $0.044 \pm 0.054$ & $0.046 \pm 0.052$ & $0.051 \pm 0.062$ & $0.026 \pm 0.070$ & $0.070 \pm 0.069$ & $0.030 \pm 0.083$ \\ \hline
			loss&$5.001 \pm 1.760$ & $5.950 \pm 1.904$ & $5.287 \pm 1.946$ & $6.140 \pm 2.033$ & $13.099 \pm 4.899$ & $5.417 \pm 1.876$ & $13.122 \pm 5.131$ \\ \hline
		\end{tabular}
	}\caption{Таблица сравнения качества по языкам для атаки Word level}
\end{table}


In [12]:
# СРАВНЕНИЕ ЯЗЫКОВ ДЛЯ АТАКИ ALIGNMENTS

output = {}

for language in config['languages']:
    results = {'intent_acc': [], 'slot_f1': [], 'sementic_frame_acc': [], 'loss': []}

    for model_name, model_arg in zip(model_names, model_args):

        df = get_model_attacks(language, *model_arg)

        for key in results.keys():
            for idx, row in df.iterrows():
                if 'Alignments' in idx:
                    results[key].append(row[key])

    output[language] = results
    
output = pd.DataFrame.from_dict(output)

output

,en,de,es,fr,ja,pt,zh
intent_acc,"[0.9178807947019868, 0.8913907284768212, 0.898...","[0.9205298013245032, 0.8860927152317881, 0.905...","[0.8940397350993378, 0.9019867549668874, 0.904...","[0.9112582781456954, 0.9258278145695364, 0.907...","[0.8940397350993378, 0.8900662251655629, 0.884...","[0.9019867549668874, 0.9112582781456954, 0.890...","[0.8874172185430463, 0.8754966887417218, 0.854..."
slot_f1,"[0.7667051430037194, 0.7720763723150358, 0.719...","[0.7573018991486575, 0.6782150776053215, 0.773...","[0.7297451719803452, 0.6389905362776024, 0.642...","[0.7028210743269354, 0.7412149665670793, 0.678...","[0.5394748170469221, 0.5529166666666667, 0.480...","[0.7504013832283563, 0.7588441803942748, 0.738...","[0.655124266964535, 0.638572746847124, 0.56930..."
sementic_frame_acc,"[0.4185430463576159, 0.3125827814569536, 0.262...","[0.3642384105960264, 0.2794701986754966, 0.350...","[0.2132450331125827, 0.1854304635761589, 0.177...","[0.2569536423841059, 0.3576158940397351, 0.229...","[0.0092715231788079, 0.0225165562913907, 0.001...","[0.3178807947019867, 0.4304635761589404, 0.308...","[0.0158940397350993, 0.0317880794701986, 0.010..."
loss,"[2.1262399385438155, 2.0077294078312424, 2.872...","[2.784261039869958, 3.218310609606928, 3.26534...","[2.78036870650555, 3.519052504081475, 3.542412...","[2.35085093065312, 2.427167996685756, 2.393987...","[10.76169678788436, 11.011205083445498, 12.290...","[2.224347427269248, 1.9758701035685493, 1.9506...","[10.492342512231124, 10.640462155091136, 11.80..."


In [13]:
print(df_to_latex(output, 'Таблица сравнения качества по языкам для атаки Alignments'))

\begin{table}[H]
	\resizebox{\textwidth}{!}{
		\begin{tabular}{|>{\bfseries}l|c|c|c|c|c|c|c|}
			\hline
			& en & de & es & fr & ja & pt & zh \\ \hline
			intent\_acc&$0.919 \pm 0.021$ & $0.913 \pm 0.024$ & $0.913 \pm 0.025$ & $0.921 \pm 0.023$ & $0.897 \pm 0.020$ & $0.904 \pm 0.029$ & $0.890 \pm 0.022$ \\ \hline
			slot\_f1&$0.680 \pm 0.142$ & $0.661 \pm 0.143$ & $0.598 \pm 0.147$ & $0.636 \pm 0.136$ & $0.563 \pm 0.068$ & $0.658 \pm 0.143$ & $0.630 \pm 0.051$ \\ \hline
			sementic\_frame\_acc&$0.305 \pm 0.137$ & $0.302 \pm 0.116$ & $0.192 \pm 0.092$ & $0.271 \pm 0.117$ & $0.037 \pm 0.058$ & $0.322 \pm 0.151$ & $0.054 \pm 0.079$ \\ \hline
			loss&$2.949 \pm 1.592$ & $3.291 \pm 0.956$ & $3.628 \pm 1.548$ & $2.729 \pm 0.828$ & $10.237 \pm 1.783$ & $2.683 \pm 1.314$ & $9.786 \pm 2.084$ \\ \hline
		\end{tabular}
	}\caption{Таблица сравнения качества по языкам для атаки Alignments}
\end{table}
